In [6]:
import re
import tensorflow as tf

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
data_path = tf.keras.utils.get_file("shakespeare.txt", origin=url)

# Read once, do light cleanup outside the pipeline
raw = tf.io.read_file(data_path).numpy().decode("utf-8")
text = raw.lower()
text = re.sub(r"\[.*?\]", "", text)      # drop stage directions
text = re.sub(r"\s+", " ", text).strip() # collapse whitespace


In [7]:
chars = sorted(set(text))
vocab_size = len(chars)
keys = tf.constant(chars)
vals = tf.range(vocab_size, dtype=tf.int64)

table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(keys, vals),
    default_value=0
)


In [8]:
seq_len, batch_size, buffer = 100, 64, 10_000

ds = (
    tf.data.Dataset
      .from_tensor_slices(tf.strings.unicode_split(text, "UTF-8"))
      .map(table.lookup, num_parallel_calls=tf.data.AUTOTUNE)
      # sliding window of length+1, 1‑step shift
      .window(seq_len+1, shift=1, drop_remainder=True)
      .flat_map(lambda w: w.batch(seq_len+1))
      .map(lambda w: (w[:-1], w[1:]), num_parallel_calls=tf.data.AUTOTUNE)
      .shuffle(buffer)
      .batch(batch_size, drop_remainder=True)
      .cache()                    # keep one epoch in RAM
      .repeat()                   # infinite for model.fit
      .prefetch(tf.data.AUTOTUNE) # overlap preprocessing & training
)


In [1]:
import os
from tensorflow import keras

model_path = "shakespeare_model.h5"

if os.path.exists(model_path):
    print("✅ Loading saved model...")
    model = keras.models.load_model(model_path)

else:
    print("🔧 Building new model...")
    
    embed_dim = 128
    rnn_units = 256

    model = keras.Sequential([
        keras.layers.Input(shape=[None]),
        keras.layers.Embedding(vocab_size, embed_dim),
        keras.layers.GRU(rnn_units, return_sequences=True, dropout=0.2),
        keras.layers.TimeDistributed(
            keras.layers.Dense(vocab_size, activation="softmax")
        )
    ])

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

model.summary()


2025-07-07 08:21:40.089784: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-07 08:21:40.374408: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, None, 128)      │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, None, 256)      │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, None, 38)       │         9,766 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 311,080 (1.19 MB)

 Trainable params: 311,078 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [14]:
steps = (len(text) - seq_len) // batch_size

callbacks = [
    keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2),
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint("best.h5", save_best_only=True)
]

history = model.fit(
    ds,
    epochs=1,
    steps_per_epoch=steps,
    callbacks=callbacks,
    verbose=1
)


17313/17313 ━━━━━━━━━━━━━━━━━━━━ 3156s 182ms/step - accuracy: 0.7227 - loss: 0.9355 - learning_rate: 0.0010


In [22]:
# Rebuild vocab if not done already
vocab = sorted(set(text))
id2char = np.array(vocab)  # index → character mapping


In [26]:
def generate(seed="romeo:", length=300, temp=0.8):
    chars = list(seed.lower())
    ids = table.lookup(tf.constant(chars)).numpy().tolist()
    out = chars.copy()

    for _ in range(length):
        x = tf.expand_dims(ids[-seq_len:], 0)
        probs = model.predict(x, verbose=0)[0, -1]
        nxt = sample(probs, temp)
        ids.append(nxt)
        out.append(id2char[nxt])  # fixed line

    return "".join(out)


In [29]:
print(generate("juliet:", temp=0.5))  # more predictable
print(generate("juliet:", temp=1.0))  # balanced
print(generate("juliet:", temp=1.2))  # more creative


juliet: hark! would i had never married my daughter there! gonzalo: all things in common nature should produce of surpet it underge and heart. antonio: the fire. petruchio: and then to be reputal more, and she didons a shorsed with good the subjects? antonio: none, man; all idle, whom i prithee, peace, my 
juliet: do not thy stowam, all, sleep. gonzalo: when i wores are in either's poxting into my this is at your sirst gonzalo: by 'longing singerity, would it not laft mine, and mille! antonio: if but are two oft? ariel: she saw, my father's laft noor on a his merrive to comfort thee: for, come, thou shouldst
juliet: houth. ariel: so shalt the dind he. spoken, do commits you speak? gonzalo: what is't contitait:. for we twints, is not this blesserfy. slave, should pardon, my remembant comfort like conscien. sebastian: as siching me thy beheod this to sleep, and such perses! o, i flot the most, in disp; o hont, w


In [31]:
model.save("shakespeare_model.h5")  # HDF5 format
